# boardデータのsample

In [50]:
import sqlite3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Connect to the SQLite database
# conn = sqlite3.connect(r'H:\マイドライブ\pytest\virtual_currency\gmo\gmo_data\board\2023-08-15_board2.db')
conn = sqlite3.connect(r'C:\Users\yamaguchi\MyDocument\gmo_data\board\2023-09-16\2023-09-16-0_board.db')

# Query the data
board_data_df = pd.read_sql_query("SELECT * FROM board_data", conn)
asks_data_df = pd.read_sql('SELECT * FROM asks', conn)
bids_data_df = pd.read_sql('SELECT * FROM bids', conn)

# Convert 'responsetime' in board_data and 'board_data_responsetime' in asks and bids to datetime format
board_data_df['responsetime'] = pd.to_datetime(board_data_df['responsetime'])
asks_data_df['board_data_responsetime'] = pd.to_datetime(asks_data_df['board_data_responsetime'])
bids_data_df['board_data_responsetime'] = pd.to_datetime(bids_data_df['board_data_responsetime'])


In [6]:
asks_data_df.head()

,id,price,size,board_data_responsetime
0,1,3891950.0,0.100,2023-09-15 23:59:59.997
1,2,3891998.0,0.150,2023-09-15 23:59:59.997
2,3,3891999.0,0.110,2023-09-15 23:59:59.997
3,4,3892048.0,0.010,2023-09-15 23:59:59.997
4,5,3892049.0,0.077,2023-09-15 23:59:59.997


In [9]:
sample_responsetime = list(board_data_df['responsetime'].head(60).values)
sample_responsetime

[numpy.datetime64('2023-09-15T23:59:59.997000000'),
 numpy.datetime64('2023-09-16T00:00:02.006000000'),
 numpy.datetime64('2023-09-16T00:00:03.011000000'),
 numpy.datetime64('2023-09-16T00:00:04.017000000'),
 numpy.datetime64('2023-09-16T00:00:04.519000000'),
 numpy.datetime64('2023-09-16T00:00:06.028000000'),
 numpy.datetime64('2023-09-16T00:00:06.531000000'),
 numpy.datetime64('2023-09-16T00:00:08.541000000'),
 numpy.datetime64('2023-09-16T00:00:10.048000000'),
 numpy.datetime64('2023-09-16T00:00:10.551000000'),
 numpy.datetime64('2023-09-16T00:00:12.059000000'),
 numpy.datetime64('2023-09-16T00:00:13.063000000'),
 numpy.datetime64('2023-09-16T00:00:13.566000000'),
 numpy.datetime64('2023-09-16T00:00:15.575000000'),
 numpy.datetime64('2023-09-16T00:00:17.084000000'),
 numpy.datetime64('2023-09-16T00:00:17.586000000'),
 numpy.datetime64('2023-09-16T00:00:18.591000000'),
 numpy.datetime64('2023-09-16T00:00:19.596000000'),
 numpy.datetime64('2023-09-16T00:00:20.601000000'),
 numpy.datet

In [10]:
sample_responsetime = list(board_data_df['responsetime'].head(60).values)

new_conn = sqlite3.connect('sample.db')

for df, name in zip([board_data_df, asks_data_df, bids_data_df], ['board_data', 'asks', 'bids']):
    if name != "board_data":
        sample = df[df['board_data_responsetime'].isin(sample_responsetime)]
    else:
        sample = df[df['responsetime'].isin(sample_responsetime)]
    sample.to_sql(name, new_conn, index=False, if_exists='replace')
new_conn.close()

# ratest_rateのsample

In [22]:
import sqlite3
import pandas as pd
import numpy as np

start_time = pd.Timestamp(np.datetime64(sample_responsetime[0]))
end_time = pd.Timestamp(np.datetime64(sample_responsetime[-1]))

start_time, end_time

(Timestamp('2023-09-15 23:59:59.997000'),
 Timestamp('2023-09-16 00:01:09.843000'))

In [28]:


rate_conn = sqlite3.connect(r'C:\Users\yamaguchi\MyDocument\gmo_data\board\2023-09-16\2023-09-16_ratest_rate.db')
trading_hist_conn = sqlite3.connect(r'C:\Users\yamaguchi\MyDocument\gmo_data\board\2023-09-16\2023-09-16_trading_hist.db')

rate_df = pd.read_sql('SELECT * FROM ratest_rate', rate_conn)
rate_df['timestamp'] = pd.to_datetime(rate_df['timestamp'])

trading_hist_df = pd.read_sql('SELECT * FROM trading_history', trading_hist_conn)
trading_hist_df['timestamp'] = pd.to_datetime(trading_hist_df['timestamp'])

rate_df.head(), trading_hist_df.head()

(   id               timestamp channel      ask      bid     high     last  \
 0   1 2023-09-16 00:00:14.367  ticker  3891598  3889800  3950000  3891598   
 1   2 2023-09-16 00:00:16.767  ticker  3891450  3888500  3950000  3889800   
 2   3 2023-09-16 00:00:16.767  ticker  3891450  3888500  3950000  3888500   
 3   4 2023-09-16 00:00:33.754  ticker  3891863  3889601  3950000  3891863   
 4   5 2023-09-16 00:01:36.059  ticker  3894200  3894001  3950000  3894001   
 
        low symbol    column  
 0  3887506    BTC  122.6126  
 1  3887506    BTC  122.6326  
 2  3887506    BTC  122.6326  
 3  3887506    BTC  122.3736  
 4  3887506    BTC  122.0228  ,
    id               timestamp    price channel    size  side column
 0   1 2023-09-16 00:00:14.367  3891598  trades  0.0100   BUY    BTC
 1   2 2023-09-16 00:00:16.767  3889800  trades  0.0005  SELL    BTC
 2   3 2023-09-16 00:00:16.767  3888600  trades  0.0050  SELL    BTC
 3   4 2023-09-16 00:00:16.767  3888500  trades  0.0045  SELL    BT

In [29]:
# 時間範囲内のデータを抽出
rate_df_sample = rate_df[(rate_df['timestamp'] > start_time) & (rate_df['timestamp'] < end_time)]
trading_hist_df_sample = trading_hist_df[(trading_hist_df['timestamp'] > start_time) & (trading_hist_df['timestamp'] < end_time)]

In [30]:
rate_df_sample.to_sql()

,id,timestamp,channel,ask,bid,high,last,low,symbol,column
0,1,2023-09-16 00:00:14.367,ticker,3891598,3889800,3950000,3891598,3887506,BTC,122.6126
1,2,2023-09-16 00:00:16.767,ticker,3891450,3888500,3950000,3889800,3887506,BTC,122.6326
2,3,2023-09-16 00:00:16.767,ticker,3891450,3888500,3950000,3888500,3887506,BTC,122.6326
3,4,2023-09-16 00:00:33.754,ticker,3891863,3889601,3950000,3891863,3887506,BTC,122.3736


In [31]:
trading_hist_df_sample

,id,timestamp,price,channel,size,side,column
0,1,2023-09-16 00:00:14.367,3891598,trades,0.0100,BUY,BTC
1,2,2023-09-16 00:00:16.767,3889800,trades,0.0005,SELL,BTC
2,3,2023-09-16 00:00:16.767,3888600,trades,0.0050,SELL,BTC
3,4,2023-09-16 00:00:16.767,3888500,trades,0.0045,SELL,BTC
4,5,2023-09-16 00:00:33.754,3891863,trades,0.0001,BUY,BTC


In [ ]:
rate_df_sample_conn = sqlite3.connect('rate_df_sample.db')
trading_hist_df_sample_conn = sqlite3.connect('trading_hist_df_sample.db')



In [37]:
trading_hist_df_sample.columns

Index(['id', 'timestamp', 'price', 'channel', 'size', 'side', 'column'], dtype='object')

In [ ]:
特徴量エンジニアリングの提案
時間帯ごとの集計: board_data, asks, bids からは1分または5分ごとに価格とサイズの平均、最大値、最小値、標準偏差などを計算できます。

レートと取引履歴の統合: ratest_rate と trading_history のデータを時間帯ごとに集計し、これを特徴量として追加できます。例えば、各時間帯での平均取引価格、取引量、最高・最低価格など。

時系列特徴量: 過去N分間の価格変動、過去N分間の取引量など、時間的な側面を考慮した特徴量も有用かもしれません。

テクニカル指標: 移動平均、ボリンジャーバンド、RSI（相対強弱指数）などのテクニカル指標を計算して特徴量として使用することも考えられます。


ok，概ねいい感じです．
まず，asksとbidsについて取り扱おう．board_dataは考慮すべき事柄が多いので後回しにする．
これらのデータに対して，提案した特徴量エンジニアリングを行う．
1.時間帯ごとの集計: asks, bids からは1分と5分ごとに価格とサイズの平均、最大値、最小値、標準偏差などを計算して，それをpdにまとめたい．

In [ ]:
ratest_rate
'id','timestamp', 'channel', 'ask', 'bid', 'high', 'last', 'low', 'symbol', 'volume'
1,2023-09-16 00:00:16.767000,ticker,3891450,3888500,3950000,3889800,3887506,BTC,122.6326
2,2023-09-16 00:00:16.767000,ticker,3891450,3888500,3950000,3888500,3887506,BTC,122.6326

trading_history
'id', 'timestamp', 'price', 'channel', 'size', 'side', 'symbol'
1,2023-09-16 00:00:16.767000,3889800,trades,0.0005,SELL,BTC
2,2023-09-16 00:00:16.767000,3888600,trades,0.005,SELL,BTC
3,2023-09-16 00:00:16.767000,3888500,trades,0.0045,SELL,BTC

In [ ]:
new_data
'id', 'timestamp', 'price', 'channel', 'size', 'side', 'symbol', 'ask', 'bid', 'high', 'low', 
1,2023-09-16 00:00:16.767000,3889800,trades,0.0005,SELL,BTC
2,2023-09-16 00:00:16.767000,3888600,trades,0.005,SELL,BTC
3,2023-09-16 00:00:16.767000,3888500,trades,0.0045,SELL,BTC

In [38]:
# Reconnect to the uploaded SQLite databases to read the data
conn_rate = sqlite3.connect(r'C:\Users\yamaguchi\MyDocument\gmo_data\board\2023-09-16\2023-09-16_ratest_rate.db')
conn_trading_hist = sqlite3.connect(r'C:\Users\yamaguchi\MyDocument\gmo_data\board\2023-09-16\2023-09-16_trading_hist.db')

# Read the data from ratest_rate and trading_history tables
ratest_rate_data = pd.read_sql('SELECT * FROM ratest_rate', conn_rate)
trading_hist_data = pd.read_sql('SELECT * FROM trading_history', conn_trading_hist)

# Close the connections
conn_rate.close()
conn_trading_hist.close()

# Count the number of occurrences for each timestamp in both datasets
ratest_rate_counts = ratest_rate_data['timestamp'].value_counts().reset_index()
ratest_rate_counts.columns = ['timestamp', 'ratest_rate_count']

trading_hist_counts = trading_hist_data['timestamp'].value_counts().reset_index()
trading_hist_counts.columns = ['timestamp', 'trading_hist_count']

# Merge the counts based on timestamp
merged_counts = pd.merge(ratest_rate_counts, trading_hist_counts, on='timestamp', how='inner')

# Filter rows where the counts are not equal
unequal_counts = merged_counts[merged_counts['ratest_rate_count'] != merged_counts['trading_hist_count']]

unequal_counts.head()


,timestamp,ratest_rate_count,trading_hist_count
0,2023-09-16 08:04:29.511000,2,3
1,2023-09-16 07:52:12.323000,2,6
2,2023-09-16 07:39:53.133000,2,6
4,2023-09-16 08:00:06.979000,2,7
9,2023-09-16 07:39:51.323000,2,3


In [49]:
# timestamp列の値を集合として取得
ratest_rate_timestamps = set(ratest_rate_data['timestamp'])
trading_hist_timestamps = set(trading_hist_data['timestamp'])

# ratest_rate_dataにはあるがtrading_hist_dataにはないタイムスタンプ
unique_to_ratest = ratest_rate_timestamps - trading_hist_timestamps

# trading_hist_dataにはあるがratest_rate_dataにはないタイムスタンプ
unique_to_trading = trading_hist_timestamps - ratest_rate_timestamps

# 結果を表示（または任意の処理）
print("Unique to ratest_rate_data:", unique_to_ratest)
print("Unique to trading_hist_data:", unique_to_trading)

Unique to ratest_rate_data: {'2023-09-16 09:00:13.927000', '2023-09-16 07:53:06.661000', '2023-09-16 11:10:05.538000', '2023-09-16 07:49:54.171000', '2023-09-16 07:53:06.548000', '2023-09-16 11:00:05.053000', '2023-09-16 08:04:56.577000'}
Unique to trading_hist_data: set()


# rateest_rateの中のvolumeがboardのデータから導けるか調べる．

In [51]:
import sqlite3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Connect to the SQLite database
# conn = sqlite3.connect(r'H:\マイドライブ\pytest\virtual_currency\gmo\gmo_data\board\2023-08-15_board2.db')
conn = sqlite3.connect(r'C:\Users\yamaguchi\MyDocument\gmo_data\board\2023-09-16\2023-09-16-0_board.db')

# Query the data
board_data_df = pd.read_sql_query("SELECT * FROM board_data", conn)
asks_data_df = pd.read_sql('SELECT * FROM asks', conn)
bids_data_df = pd.read_sql('SELECT * FROM bids', conn)

# Convert 'responsetime' in board_data and 'board_data_responsetime' in asks and bids to datetime format
board_data_df['responsetime'] = pd.to_datetime(board_data_df['responsetime'])
asks_data_df['board_data_responsetime'] = pd.to_datetime(asks_data_df['board_data_responsetime'])
bids_data_df['board_data_responsetime'] = pd.to_datetime(bids_data_df['board_data_responsetime'])

In [59]:
asks_data_df.drop(['id', 'price'], axis = 1).groupby('board_data_responsetime').sum().head(20)

,size
board_data_responsetime,
2023-09-15 23:59:59.997,66.7751
2023-09-16 00:00:02.006,67.5997
2023-09-16 00:00:03.011,66.0234
2023-09-16 00:00:04.017,65.9783
2023-09-16 00:00:04.519,65.9128
2023-09-16 00:00:06.028,65.7254
2023-09-16 00:00:06.531,65.6339
2023-09-16 00:00:08.541,66.0605
2023-09-16 00:00:10.048,66.9165


In [60]:
bids_data_df.drop(['id', 'price'], axis = 1).groupby('board_data_responsetime').sum().head(20)

,size
board_data_responsetime,
2023-09-15 23:59:59.997,73.5841
2023-09-16 00:00:02.006,77.5210
2023-09-16 00:00:03.011,77.5171
2023-09-16 00:00:04.017,76.8911
2023-09-16 00:00:04.519,75.1212
2023-09-16 00:00:06.028,75.0133
2023-09-16 00:00:06.531,75.2121
2023-09-16 00:00:08.541,74.9677
2023-09-16 00:00:10.048,76.0397


In [58]:
asks_data_df.drop('id', axis=1)

,price,size,board_data_responsetime
0,3891950.0,0.1000,2023-09-15 23:59:59.997
1,3891998.0,0.1500,2023-09-15 23:59:59.997
2,3891999.0,0.1100,2023-09-15 23:59:59.997
3,3892048.0,0.0100,2023-09-15 23:59:59.997
4,3892049.0,0.0770,2023-09-15 23:59:59.997
...,...,...,...
1553495,4140000.0,0.0788,2023-09-16 00:59:59.752
1553496,4140600.0,0.0010,2023-09-16 00:59:59.752
1553497,4141199.0,0.0001,2023-09-16 00:59:59.752
1553498,4141238.0,0.0010,2023-09-16 00:59:59.752
